In [ ]:
from azure.ml import MLClient
from azure.ml.entities import CommandJob
from azure.identity import InteractiveBrowserCredential

In [ ]:
#Enter details of your AML workspace
subscription_id = 'b17253fa-f327-42d6-9686-f3e553e24763'
resource_group = 'testharnvir-local'
workspace = 'diab-ws01'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

In [ ]:
#create the command job
job = CommandJob(
    code_local_path="./src", #local path where the code is stored
    command = "python main.py --epochs ${{inputs.epochs}} --model-dir ${{inputs.model_dir}}",
    inputs={"epochs":1, "model_dir":"outputs/keras-model"},
    environment = "AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu:1",
    compute = "cpu-cluster",
    instance_count=2,
    # distribution = {"distribution_type": "Mpi", "process_count_per_instance": 1}, 
    distribution = {
        "distribution_type": "TensorFlow",
        "parameter_server_count": 1,
        "worker_count": 2,
        "added_property": 7,
    },
    # distribution = {
    #        "distribution_type": "PyTorch",
    #        "process_count_per_instance": 4,
    #        "additional_prop": {"nested_prop": 3},
    #    },
    display_name = "tensorflow-mnist-distributed-example"
    #experiment_name: tensorflow-mnist-distributed-example
    #description: Train a basic neural network with TensorFlow on the MNIST dataset, distributed via TensorFlow.
)

In [ ]:
#submit the command job
returned_job = ml_client.create_or_update(job)
#get a URL for the status of the job
returned_job.services["Studio"].endpoint